In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from sklearn.decomposition import TruncatedSVD
# If nltk stop word is not downloaded
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
import nltk,csv,numpy
nltk.download('punkt')
from nltk import sent_tokenize, word_tokenize, pos_tag
sentences=[]
reader = csv.reader(open('/content/drive/My Drive/instacomments.csv', 'rU'), delimiter= ",",quotechar='|')
for line in reader:
    for field in line:
      sentences.append(field)
print(sentences)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: 'U' mode is deprecated
  """


['user name', 'comments', 'madhura_makeupnhair', 'Can’t wait for season 2!!!', '"battatawada', 'Verified"', 'Verified', 'abby_racer20', '@harshita1210 Miss Sanyukta.. wat abt Sadda Haq??', 'mirahul03', 'Kab aayegi मिर्ज़ापुर', 'harshitasinha122', 'Yayyy 🤗🤗🤗', 'nikkhiillllll', '@divyenndu', 'samannawaj123', 'I am Big Fan of Mirzapur Series', '_beingasifkhan', 'Aur kitna wait karaoge yrr tum log abhi bhabhi or raja ka Scene dekh ke kaam chalana padta hai', 'harshitasinha122', '💞', 'harshitasinha122', '👌👌👌', 'harshitasinha122', '👌👌👌', 'krishna_akkian', '25 September may be', 'siyasingh2801', '😍😍', 'siyasingh2801', 'Wai', 'siyasingh2801', 'Waiting baby❤️❤️', 'siyasingh2801', 'Plz be safe love....take care of yourself❤️❤️', 'sagarnitesh55', 'Oh my god😍😘 you are so lovely !❤️', 'harsh_k_pvt', 'Most awating❤️😭', 'iamsudipamanna', '❤️', 'dharmesh.dy', 'Excited 😍', 'krishnagopal.paul_official', '"*Do you want to Any kind of portrait or painting wall painting ect. ?* click on the below *link*__ 

In [4]:
df = pd.DataFrame()
df["documents"] = sentences
df.head()
df['clean_documents'] = df['documents'].str.replace("[^a-zA-Z#]", " ")
df['clean_documents'] = df['clean_documents'].fillna('').apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))
df['clean_documents'] = df['clean_documents'].fillna('').apply(lambda x: x.lower())

df.head()

,documents,clean_documents
0,user name,user name
1,comments,comments
2,madhura_makeupnhair,madhura makeupnhair
3,Can’t wait for season 2!!!,can wait for season
4,"""battatawada",battatawada


In [ ]:
# tokenization
tokenized_doc = df['clean_documents'].fillna('').apply(lambda x: x.split())

# remove stop-words

tokenized_doc = tokenized_doc.apply(lambda x: [itemstop_words = stopwords.words('english')
 for item in x if item not in stop_words])

# de-tokenization
detokenized_doc = []
for i in range(len(df)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

df['clean_documents'] = detokenized_doc



In [7]:
# TF-IDF vector
vectorizer = TfidfVectorizer(stop_words='english', smooth_idf=True)
X = vectorizer.fit_transform(df['clean_documents'])
X.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [8]:
# SVD represent documents and terms in vectors 
svd_model = TruncatedSVD(n_components=2, algorithm='randomized', n_iter=100, random_state=122)
lsa = svd_model.fit_transform(X)

#Documents - Topic vector
pd.options.display.float_format = '{:,.16f}'.format
topic_encoded_df = pd.DataFrame(lsa, columns = ["topic_1", "topic_2"])
topic_encoded_df["documents"] = df['clean_documents']
display(topic_encoded_df[["documents", "topic_1", "topic_2"]])

,documents,topic_1,topic_2
0,user name,-0.0000000000000000,-0.0000000000000000
1,comments,0.0000000000000000,0.0000000000000000
2,madhura makeupnhair,-0.0000000000000000,-0.0000000000000000
3,can wait for season,-0.0000000000000009,0.0300069253896707
4,battatawada,0.0000000000000000,0.0000000000000000
...,...,...,...
268,,-0.0000000000000000,0.0000000000000000
269,urimrannn,0.0000000000000000,-0.0000000000000000
270,,-0.0000000000000000,0.0000000000000000
271,urimrannn,0.0000000000000000,-0.0000000000000000


In [9]:
dictionary = vectorizer.get_feature_names()

encoding_matrix = pd.DataFrame(svd_model.components_, index = ["topic_1","topic_2"], columns = (dictionary)).T
encoding_matrix

,topic_1,topic_2
aakhir,-0.0000000000000002,0.0019049074196449
aandolan,-0.0000000000000000,0.0008784362662326
aap,-0.0000000000000000,0.0014066164190548
aapki,-0.0000000000000000,0.0019049074196449
aapko,-0.0000000000000000,0.0002411070089272
...,...,...
youth,-0.0000000000000000,-0.0000000000000000
yrr,-0.0000000000000000,0.0012063529114341
yrrr,-0.0000000000000005,0.0201925598465197
zaida,-0.0000000000000000,0.0014416511410439
